In [1]:
# Dependency conflicts
!pip install dask==2021.4.0 sparse==0.9.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 941.1/941.1 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 151.8 MB/s eta 0:00:00
  Attempting uninstall: sparse
    Found existing installation: sparse 0.15.1
    Uninstalling sparse-0.15.1:
      Successfully uninstalled sparse-0.15.1
  Attempting uninstall: dask
    Found existing installation: dask 2023.11.0
    Uninstalling dask-2023.11.0:
      Successfully uninstalled dask-2023.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2023.11.0 requires dask==2023.11.0, but you have dask 2021.4.0 which is incompatible.
rapids-dask-dependency 23.12.1 requires dask==2023.11.0, but you have dask 2021.4.0 which is incompatible.


In [1]:
import os
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from scipy.sparse import csr_matrix, issparse, csc_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox
import decoupler as dc
import rpy2.robjects as ro
import diffxpy.api as de
import batchglm
import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

from batchglm.pkg_constants import TF_CONFIG_PROTO

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython
os.chdir('/root/host_home')


2025-07-18 09:02:42.726200: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import session_info
session_info.show(excludes=['pybind11_abseil'])

In [4]:
%%R
library(edgeR)
library(scater)
library(MAST)
library(Seurat)
library(DESeq2)
library(data.table)
library(dplyr)
#library(EnhancedVolcano)
library(apeglm)
library(lme4)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: limma
Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2

In [5]:
%%R
sessionInfo()

R version 4.2.3 (2023-03-15)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 22.04.3 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/atlas/libblas.so.3.10.3
LAPACK: /usr/lib/x86_64-linux-gnu/atlas/liblapack.so.3.10.3

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats4    tools     stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] lme4_1.1-35.2               Matrix_1.6-5               
 [3] apeglm_1.20.0               dplyr_1.1.4                
 [5] data.table_1.15.4           DESeq2_1.38.3              
 [7] Seurat_5.0.3                SeuratObject_5.0.1         
 [9] sp_2.1-3                    MAST_1.24.1            

In [3]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
#sc.logging.print_versions()



#logging.getLogger("tensorflow").setLevel(logging.ERROR)
#logging.getLogger("batchglm").setLevel(logging.INFO)


pd.set_option('display.max_rows', 500)
#warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")

In [4]:
#Set number of threads
TF_CONFIG_PROTO.inter_op_parallelism_threads = 1
TF_CONFIG_PROTO.intra_op_parallelism_threads = 12

# Filenames

In [5]:
#input
adata_input = 'lp_NPCs/processed_data/lp_NPC_clustered_scvi_cuda.h5ad'

In [6]:
#output
save_dir = 'lp_NPCs/tables/' 
resolution = 'cell_types'

# Read in data

In [7]:
adata = sc.read(adata_input)


In [8]:
adata

AnnData object with n_obs × n_vars = 30843 × 17250
    obs: 'Donor', 'lane', 'condition', 'age', 'years_of_illness', 'deltaHY', 'deltaUPDRS', 'deltaADL', 'deltaDOPA', 'gender', 'cultivation_period', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_counts', 'doublet_score', 'size_factors', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'Ferroptosis_pos_corr', 'Ferroptosis_neg_corr', 'Apoptosis_pos_corr', 'Apoptosis_neg_corr', 'Necroptosis_pos_corr', 'Autophagy_pos_corr', 'Pathanatos_pos_corr', 'cell_types', 'cell_types_abbrev'
    var: 'gene_ids', 'feature_types', 'genome', 'gene_symbols', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_genes', 'highly_variable_nbatches', '

# Preprocess data

In [9]:
# Clean up cell type names
adata.obs['cell_types_used'] = [ct.replace(" ", "-") for ct in adata.obs[resolution]]
adata.obs['cell_types_used'] = [ct.replace("+", "") for ct in adata.obs['cell_types_used']]

In [10]:
adata.obs["Donor"] = adata.obs["Donor"].astype("category")
adata.obs["condition"] = adata.obs["condition"].astype("category")
adata.obs['cell_types_used'] = adata.obs['cell_types_used'].astype("category")

In [11]:
adata.obs['cell_types_used'].cat.categories

Index(['Apop.-NCSC', 'Apop.-NSC', 'Glial-precursors', 'Immature-neurons',
       'NCSC', 'NSC1a', 'NSC1b', 'NSC2a', 'NSC2b'],
      dtype='object')

# Check the data

In [12]:
np.mean(adata.layers['soupX_counts'].astype(int) != adata.layers['soupX_counts'])

0.0

Counts are actually all integer counts.

In [13]:
adata.obs.condition.value_counts()

condition
Parkinson    19145
Control      11698
Name: count, dtype: int64

In [14]:
adata.obs.groupby(['cell_types_used'])['condition'].value_counts()

cell_types_used   condition
Apop.-NCSC        Parkinson      58
                  Control        57
Apop.-NSC         Control      1283
                  Parkinson    1198
Glial-precursors  Control       646
                  Parkinson     250
Immature-neurons  Parkinson     482
                  Control       428
NCSC              Control      1663
                  Parkinson    1197
NSC1a             Parkinson    9159
                  Control      4821
NSC1b             Parkinson    4391
                  Control      2549
NSC2a             Parkinson    2015
                  Control       213
NSC2b             Parkinson     395
                  Control        38
Name: count, dtype: int64

In [15]:
adata.obs.groupby(['cell_types_used'])['Donor'].value_counts()

cell_types_used   Donor          
Apop.-NCSC        UKERiO3H-R1-005      31
                  UKERi82A-R1-002      17
                  UKERiAY6-R1-003      15
                  UKERiM89-R1-005       9
                  UKERiPX7-R1-001       9
                  UKERi88H-R1-002       9
                  UKERiC99-R1-007       8
                  UKERiJ2C-R1-015       6
                  UKERiG3G-R1-039       4
                  UKERi1JF-R1-018       3
                  UKERi1E4-R1-003       2
                  UKERiR66-R1-007       2
Apop.-NSC         UKERiPX7-R1-001     447
                  UKERiO3H-R1-005     382
                  UKERi82A-R1-002     328
                  UKERi1E4-R1-003     231
                  UKERiG3G-R1-039     201
                  UKERiM89-R1-005     159
                  UKERi88H-R1-002     158
                  UKERi1JF-R1-018     141
                  UKERiAY6-R1-003     133
                  UKERiJ2C-R1-015     129
                  UKERiR66-R1-007      93


Note:
- General imbalance between Parkinson and control cell numbers


# Single-cell specific using diffxpy

In [9]:
if not os.path.exists(save_dir+'diffxpy'):
    # Create a new directory because it does not exist
    os.mkdir(save_dir+'diffxpy')
    print("The new directory: "+save_dir+"diffxpy is created!")
else: 
    print("The directory already exists")

The directory already exists


## Differential expression testing

In [17]:
de_results = dict()


In [18]:
de.utils.preview_coef_names(
    sample_description=adata.obs,
    formula="~1+condition+Donor", 
)

['Intercept',
 'condition[T.Parkinson]',
 'Donor[T.UKERiG3G-R1-039]',
 'Donor[T.UKERi1E4-R1-003]',
 'Donor[T.UKERiO3H-R1-005]',
 'Donor[T.UKERi82A-R1-002]',
 'Donor[T.UKERiJ2C-R1-015]',
 'Donor[T.UKERiM89-R1-005]',
 'Donor[T.UKERiC99-R1-007]',
 'Donor[T.UKERiR66-R1-007]',
 'Donor[T.UKERiAY6-R1-003]',
 'Donor[T.UKERiPX7-R1-001]',
 'Donor[T.UKERi88H-R1-002]']

In [95]:
for clust in adata.obs['cell_types_used'].cat.categories:
    adata_tmp = adata[adata.obs['cell_types_used'] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs.condition.value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=50)
    print(f'Testing {adata_tmp.n_vars} genes...')
    print("")
    
    test_tmp = de.test.wald(
    data=adata_tmp.layers['soupX_counts'],
    formula_loc="~ 1 + condition + Donor + size_factors",
    as_numeric=['size_factors'],
    coef_to_test=["condition[T.Parkinson]"],
    sample_description=adata_tmp.obs,
    gene_names=adata_tmp.var['gene_ids'].astype(str),
    constraints_loc={'Donor':'condition'},
    noise_model='nb',
    dtype="float64"
    )
    
    #Store the results
    de_results[clust] = test_tmp
    

In cluster Apop.-NCSC:
condition
Parkinson    58
Control      57
Name: count, dtype: int64
filtered out 15340 genes that are detected in less than 50 cells


Built constraints: Donor[UKERi1JF-R1-018]+Donor[UKERiG3G-R1-039]+Donor[UKERi1E4-R1-003]+Donor[UKERiO3H-R1-005]+Donor[UKERi82A-R1-002]=0, Donor[UKERiJ2C-R1-015]+Donor[UKERiM89-R1-005]+Donor[UKERiC99-R1-007]+Donor[UKERiR66-R1-007]+Donor[UKERiAY6-R1-003]+Donor[UKERiPX7-R1-001]+Donor[UKERi88H-R1-002]=0


Testing 1910 genes...

training location model: True
training scale model: True
iter   0: ll=1964868.037562
iter   1: ll=1948589.106840, converged: 0.00% (loc: 0.31%, scale update: False), in 0.36sec
iter   2: ll=1944417.313854, converged: 0.00% (loc: 3.14%, scale update: False), in 0.36sec
iter   3: ll=1943961.446322, converged: 0.00% (loc: 3.14%, scale update: False), in 0.37sec
iter   4: ll=1943885.062961, converged: 0.00% (loc: 3.14%, scale update: False), in 0.35sec
iter   5: ll=1943862.272889, converged: 0.00% (loc: 9.74%, scale update: False), in 0.38sec
Fitting 1910 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter   6: ll=369925.004309, converged: 0.00% (loc: 0.00%, scale update: True), in 2.78sec
iter   7: ll=369741.548774, converged: 0.00% (loc: 4.45%, scale update: False), in 0.36sec
iter   8: ll=369721.489045, converged: 0.00% (loc: 8.69%, scale update: False), in 0.35sec
iter   9: ll=369718.612756, converged: 0.00% (loc: 24.24%, scale update: False), in 0.35sec
iter  10: ll=369717.680754, converged: 0.00% (loc: 50.10%, scale update: False), in 0.36sec
iter  11: ll=369717.354455, converged: 0.00% (loc: 65.92%, scale update: False), in 0.35sec
Fitting 1910 dispersion models: (progress not available with multiprocessing)

...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  48: ll=4918264.288686, converged: 99.96% (loc: 99.96%, scale update: True), in 2.99sec
caught 2 linalg singular matrix errors
iter  49: ll=4918264.288686, converged: 99.96% (loc: 100.00%, scale update: False), in 0.21sec
Fitting 4 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  50: ll=4918028.446200, converged: 99.98% (loc: 99.98%, scale update: True), in 2.42sec
caught 1 linalg singular matrix errors
iter  51: ll=4918028.446200, converged: 99.98% (loc: 100.00%, scale update: False), in 0.19sec
iter  52: ll=4918028.446200, converged: 100.00% (loc: 100.00%, scale update: True), in 0.18sec


/usr/local/lib/python3.10/dist-packages/dask/array/core.py:2894: RuntimeWarning: divide by zero encountered in divide
  size = (limit / dtype.itemsize / largest_block) ** (1 / len(autos))


### Bulk-like DE testing across all clusters

In [24]:
# Make a copy for the testing
adata_test = adata.copy()

print(adata_test.obs.condition.value_counts())

# Filter out genes to reduce multiple testing burden
sc.pp.filter_genes(adata_test, min_cells=50)
print(f'Testing {adata_test.n_vars} genes...')
print("")
    
test_bulk = de.test.wald(
    data=adata_test.layers['soupX_counts'],
    formula_loc="~ 1 + condition + Donor + size_factors",
    as_numeric=['size_factors'],
    coef_to_test=["condition[T.Parkinson]"],
    sample_description=adata_test.obs,
    gene_names=adata_test.var['gene_ids'].astype(str),
    constraints_loc={'Donor':'condition'},
    noise_model='nb',
    dtype="float64"
)


condition
Parkinson    19145
Control      11698
Name: count, dtype: int64
filtered out 1106 genes that are detected in less than 50 cells


Built constraints: Donor[UKERi1JF-R1-018]+Donor[UKERiG3G-R1-039]+Donor[UKERi1E4-R1-003]+Donor[UKERiO3H-R1-005]+Donor[UKERi82A-R1-002]=0, Donor[UKERiJ2C-R1-015]+Donor[UKERiM89-R1-005]+Donor[UKERiC99-R1-007]+Donor[UKERiR66-R1-007]+Donor[UKERiAY6-R1-003]+Donor[UKERiPX7-R1-001]+Donor[UKERi88H-R1-002]=0


Testing 16144 genes...



/usr/local/lib/python3.10/dist-packages/batchglm/models/base_glm/utils.py:158: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  np.mean(x[np.where(grouping == g)[0], :], axis=0)
/usr/local/lib/python3.10/dist-packages/batchglm/models/base_glm/utils.py:171: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  expect_xsq = np.vstack([

training location model: True
training scale model: True
iter   0: ll=417184428.806659
iter   1: ll=400158720.707070, converged: 0.00% (loc: 2.61%, scale update: False), in 187.91sec
iter   2: ll=396746322.719167, converged: 0.00% (loc: 3.70%, scale update: False), in 191.34sec
iter   3: ll=396324510.954030, converged: 0.00% (loc: 3.70%, scale update: False), in 187.32sec
iter   4: ll=396266031.300057, converged: 0.00% (loc: 3.88%, scale update: False), in 193.12sec
iter   5: ll=396256666.343151, converged: 0.00% (loc: 8.32%, scale update: False), in 189.98sec
Fitting 16144 dispersion models: (progress not available with multiprocessing)

...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  48: ll=385609437.939601, converged: 99.97% (loc: 99.97%, scale update: True), in 118.95sec
caught 1 linalg singular matrix errors
iter  49: ll=385609437.938112, converged: 99.97% (loc: 99.99%, scale update: False), in 2.83sec
iter  50: ll=385609437.938010, converged: 99.97% (loc: 99.99%, scale update: False), in 2.40sec
iter  51: ll=385609437.938002, converged: 99.97% (loc: 99.99%, scale update: False), in 2.31sec
iter  52: ll=385609437.938001, converged: 99.97% (loc: 99.99%, scale update: False), in 2.19sec
iter  53: ll=385609437.938001, converged: 99.97% (loc: 99.99%, scale update: False), in 2.23sec
Fitting 5 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  54: ll=385609053.956291, converged: 99.98% (loc: 99.98%, scale update: True), in 73.14sec
caught 1 linalg singular matrix errors
iter  55: ll=385609053.955791, converged: 99.98% (loc: 99.99%, scale update: False), in 2.63sec
iter  56: ll=385609053.955756, converged: 99.98% (loc: 99.99%, scale update: False), in 2.40sec
iter  57: ll=385609053.955754, converged: 99.98% (loc: 99.99%, scale update: False), in 2.43sec
iter  58: ll=385609053.955754, converged: 99.98% (loc: 99.99%, scale update: False), in 2.27sec
iter  59: ll=385609053.955754, converged: 99.98% (loc: 100.00%, scale update: False), in 2.11sec
iter  60: ll=385609053.955440, converged: 99.99% (loc: 99.99%, scale update: True), in 4.29sec
iter  61: ll=385609053.955270, converged: 99.99% (loc: 99.99%, scale update: False), in 2.30sec
iter  62: ll=385609053.955258, converged: 99.99% (loc: 99.99%, scale update: False), in 2.34sec
iter  63: ll=385609053.955257, converged: 99.99% (loc: 99.99%, scale update: False), in 2.28sec
i

/usr/local/lib/python3.10/dist-packages/dask/array/core.py:2894: RuntimeWarning: divide by zero encountered in divide
  size = (limit / dtype.itemsize / largest_block) ** (1 / len(autos))


## Check some data

In [96]:
de_results['NCSC'].summary().sort_values(by='qval', ascending=True).iloc[:10,:]

,gene,pval,qval,log2fc,mean,zero_mean,grad,coef_mle,coef_sd,ll
4925,ENSG00000106034,0.0,0.0,1.412808,0.550699,False,2.221498e-07,1.412808,0.139009,-2275.906284
2254,ENSG00000172986,0.0,0.0,1.160740,0.730769,False,5.206265e-07,1.160740,0.096420,-2835.365534
1167,ENSG00000162576,0.0,0.0,0.842886,0.351748,False,5.964570e-09,0.842886,0.097901,-1989.153244
6035,ENSG00000104332,0.0,0.0,-1.266946,8.237063,False,7.260904e-07,-1.266946,0.103472,-6232.742411
3835,ENSG00000145730,0.0,0.0,0.977643,3.631469,False,1.346294e-06,0.977643,0.059456,-5569.271067
10965,ENSG00000108821,0.0,0.0,0.983309,4.358042,False,3.514910e-07,0.983309,0.069636,-5827.515241
461,ENSG00000117425,0.0,0.0,1.302692,0.255594,False,2.540356e-07,1.302692,0.134029,-1490.484733
5360,ENSG00000128573,0.0,0.0,1.396156,1.252797,False,1.222852e-06,1.396156,0.110766,-3348.860653
13047,ENSG00000100154,0.0,0.0,0.521088,6.770280,False,3.058938e-06,0.521088,0.062448,-7458.399085
5026,ENSG00000106538,0.0,0.0,0.524391,1.863636,False,2.297171e-07,0.524391,0.060154,-5133.501959


In [97]:
# Test if model did not converger for some genes and some clusters
np.sum(de_results['NCSC'].summary()['ll'] == np.float64('-inf'))
np.sum(de_results['NSC2a'].summary()['ll'] == np.float64('-inf'))

0

0

In [25]:
test_bulk.summary().sort_values(by='qval', ascending=True).iloc[:10,:]

,gene,pval,qval,log2fc,mean,zero_mean,grad,coef_mle,coef_sd,ll
3677,ENSG00000138802,0.0,0.0,0.283842,0.949162,False,2.370641e-06,0.283842,0.017647,-36892.150601
11156,ENSG00000259132,0.0,0.0,0.393905,0.127387,False,4.817326e-07,0.393905,0.042111,-10943.588511
4112,ENSG00000109184,0.0,0.0,0.157477,0.754239,False,2.293107e-06,0.157477,0.018150,-33508.819946
4110,ENSG00000065882,0.0,0.0,0.183179,0.711701,False,1.191485e-06,0.183179,0.017381,-33023.627794
4109,ENSG00000109689,0.0,0.0,0.249432,0.661252,False,1.831842e-06,0.249432,0.019493,-31414.399101
11159,ENSG00000100804,0.0,0.0,-0.209764,7.472425,False,3.217201e-06,-0.209764,0.007050,-84480.443973
4105,ENSG00000154556,0.0,0.0,0.366729,2.324061,False,1.838497e-06,0.366729,0.018930,-53455.987897
4116,ENSG00000179299,0.0,0.0,0.284736,0.494115,False,5.652404e-07,0.284736,0.022794,-26521.401185
4103,ENSG00000169851,0.0,0.0,1.604617,1.430665,False,3.402555e-07,1.604617,0.041766,-31256.323919
4098,ENSG00000168772,0.0,0.0,0.173413,1.515806,False,6.722729e-07,0.173413,0.014631,-49341.121224


## Post-process data

In [32]:
# extract gene symbol - gene_ids informations for later conversion
df_var = pd.DataFrame(adata.var)
df_var = df_var[['gene_symbols', 'gene_ids']]
df_var.reset_index(inplace=True) 
df_var = df_var.drop(columns='index')

In [100]:
de_signif = dict()
de_full_res = dict()

for clust in de_results:
    tmp = de_results[clust].summary()
    
    # add gene symbols
    tmp = tmp.rename(columns = {"gene" : "gene_ids"})
    tmp = tmp.merge(df_var, how = 'inner', on='gene_ids')

    n_infll = np.sum(tmp['ll'] == np.float64('-inf'))
    print(f'There are {n_infll} -inf log likelihood values. Filtering these out now...')
    tmp = tmp.loc[tmp['ll'] != np.float64('-inf')]

    n_unconverged = np.sum(tmp['coef_sd'] < 10E-100)
    print(f'There are {n_unconverged} logFC std. dev.s < 10E-100. Filtering these out now...')    
    tmp = tmp.loc[tmp['coef_sd'] > 10E-100]
    
    #Sort values by q-value
    tmp.sort_values(by='qval', ascending=True, inplace=True)
    
    de_full_res[clust] = tmp.copy()
    
    #Filter by q-value to get significant DE genes
    tmp_filt = tmp.loc[tmp['qval'] < 0.05]
    de_signif[clust] = tmp_filt.copy()

There are 0 -inf log likelihood values. Filtering these out now...
There are 0 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 0 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 0 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 9 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 2 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 1 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 1 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out no

In [33]:
df_var

,gene_symbols,gene_ids
0,ATAD3B,ENSG00000160072
1,PRDM16,ENSG00000142611
2,PEX10,ENSG00000157911
3,PEX14,ENSG00000142655
4,PLCH2,ENSG00000149527
...,...,...
17245,ENSG00000276256,ENSG00000276256
17246,ENSG00000273748,ENSG00000273748
17247,ENSG00000278817,ENSG00000278817
17248,ENSG00000276345,ENSG00000276345


In [34]:
# bulk-like data
bulk_res = test_bulk.summary()

# add gene symbols
bulk_res = bulk_res.rename(columns = {"gene" : "gene_ids"})
bulk_res = bulk_res.merge(df_var, how = 'inner', on='gene_ids')

n_infll = np.sum(bulk_res['ll'] == np.float64('-inf'))
print(f'There are {n_infll} -inf log likelihood values. Filtering these out now...')
bulk_res = bulk_res.loc[bulk_res['ll'] != np.float64('-inf')]

n_unconverged = np.sum(bulk_res['coef_sd'] < 10E-100)
print(f'There are {n_unconverged} logFC std. dev.s < 10E-100. Filtering these out now...')    
bulk_res = bulk_res.loc[bulk_res['coef_sd'] > 10E-100]

#Sort values by q-value
bulk_res.sort_values(by='qval', ascending=True, inplace=True)
        
#Filter by q-value to get significant DE genes
bulk_res_filt = bulk_res.loc[bulk_res['qval'] < 0.05]


There are 0 -inf log likelihood values. Filtering these out now...
There are 2 logFC std. dev.s < 10E-100. Filtering these out now...


In [36]:
bulk_res_filt

,gene_ids,pval,qval,log2fc,mean,zero_mean,grad,coef_mle,coef_sd,ll,gene_symbols
3677,ENSG00000138802,0.000000,0.000000,0.283842,0.949162,False,2.370641e-06,0.283842,0.017647,-36892.150601,SEC24B
11156,ENSG00000259132,0.000000,0.000000,0.393905,0.127387,False,4.817326e-07,0.393905,0.042111,-10943.588511,ENSG00000259132
4112,ENSG00000109184,0.000000,0.000000,0.157477,0.754239,False,2.293107e-06,0.157477,0.018150,-33508.819946,DCUN1D4
4110,ENSG00000065882,0.000000,0.000000,0.183179,0.711701,False,1.191485e-06,0.183179,0.017381,-33023.627794,TBC1D1
4109,ENSG00000109689,0.000000,0.000000,0.249432,0.661252,False,1.831842e-06,0.249432,0.019493,-31414.399101,STIM2
...,...,...,...,...,...,...,...,...,...,...,...
5021,ENSG00000146147,0.033197,0.049792,-0.307843,0.014590,False,6.770202e-08,-0.307843,0.144548,-1973.123842,MLIP
3342,ENSG00000163507,0.033199,0.049792,0.041305,0.805272,False,1.285232e-06,0.041305,0.019395,-33569.281241,CIP2A
9859,ENSG00000067057,0.033277,0.049905,-0.054383,0.350063,False,7.901601e-07,-0.054383,0.025547,-22082.805720,PFKP
8202,ENSG00000125485,0.033295,0.049923,0.059441,0.236877,False,6.746924e-07,0.059441,0.027926,-17219.820118,DDX31


## Show DEGs

In [101]:
# Number of differentially expressed genes (DEGs)
print('DEGs at FDR < 0.05:')
{clust:de_signif[clust].shape[0] for clust in de_signif}

print('DEGs at FDR < 0.01:')
{clust:de_signif[clust].loc[(de_signif[clust]['qval']<0.01)].shape[0] for clust in de_signif}

DEGs at FDR < 0.05:


{'Apop.-NCSC': 1,
 'Apop.-NSC': 854,
 'Glial-precursors': 229,
 'Immature-neurons': 117,
 'NCSC': 2675,
 'NSC1a': 7377,
 'NSC1b': 6930,
 'NSC2a': 132,
 'NSC2b': 147}

DEGs at FDR < 0.01:


{'Apop.-NCSC': 1,
 'Apop.-NSC': 561,
 'Glial-precursors': 90,
 'Immature-neurons': 63,
 'NCSC': 1787,
 'NSC1a': 6065,
 'NSC1b': 5547,
 'NSC2a': 50,
 'NSC2b': 50}

In [28]:
#DEGs in bulk-like data
print('DEGs at FDR < 0.05:')
print(f'Bulk: {bulk_res_filt.shape[0]}')

print('DEGs at FDR < 0.01:')
degnum001 = bulk_res_filt.loc[(bulk_res_filt['qval']<0.01)].shape[0]
print(f'Bulk: {degnum001}')

DEGs at FDR < 0.05:
Bulk: 10765
DEGs at FDR < 0.01:
Bulk: 9623


Notes:
- there are quite a lot of DEGs at FDR < 0.05 and at FDR < 0.01
- Could use either threshold, FDR < 0.01 would be okay for higher confidence

## Save data

In [29]:
import pickle

In [103]:
#Pickle the de output
filename_pickle = save_dir+'diffxpy/'+resolution+'_diffxpy_cell_types_sf.pkl'
pickle.dump(de_results, open(filename_pickle, "wb"))

In [104]:
#Output individual results
for clust in de_signif:
    
    filename_signif = save_dir+'diffxpy/'+resolution+'_diffxpy_'+clust.replace(' ', '_').replace(
        '/','_').replace('(','').replace(')','')+'_diff_res.csv'
    filename_full = save_dir+'diffxpy/'+resolution+'_diffxpy_'+clust.replace(' ', '_').replace(
        '/','_').replace('(','').replace(')','')+'_res-table.csv'
    
    de_signif[clust].to_csv(filename_signif)
    de_full_res[clust].to_csv(filename_full)

In [30]:
#Pick the DE output - Bulk-like
de_output_pickle_bulk = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_sf.pkl'
pickle.dump(test_bulk, open(de_output_pickle_bulk, "wb"))

In [37]:
#Output individual results - Bulk-like

filename_signif = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_like_diff_res.csv'
filename_full = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_like_res-table.csv'
    
bulk_res_filt.to_csv(filename_signif)
bulk_res.to_csv(filename_full)

In [2]:
!pip install dask==2023.11.0 sparse==0.15.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.7 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: sparse
    Found existing installation: sparse 0.9.1
    Uninstalling sparse-0.9.1:
      Successfully uninstalled sparse-0.9.1
  Attempting uninstall: dask
    Found existing installation: dask 2021.4.0
    Uninstalling dask-2021.4.0:
      Successfully uninstalled dask-2021.4.0
